# Segmentation Modules for SPEX Pipeline

### Functions for image preproessing, segmentation, and feature extraction

In [1]:
import os
import glob
import numpy as np
from skimage import segmentation
from tifffile import TiffWriter, TiffFile

import spex_segment as sp

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
2021-07-08 14:37:35,761 [INFO] WRITING LOG OUTPUT TO /gstore/home/jesudasr/.cellpose/run.log


optimizer_v2.py (374): The `lr` argument is deprecated, use `learning_rate` instead.


In [3]:
os.chdir('/gne/data/pathology/t3imagedata/Projects/H2020-428/Images/PAC0002_KerenMIBI/Converted')
img='TA459_multipleCores2_Run-4_Point1.tiff'

### Step 1: Load image and extract channels as a list
##### (or use OMERO api to pull tiff image and channel names)

In [4]:
Image, channel=sp.load_tiff(img,is_mibi=True)

print("Image loaded with the following",len(channel),"channels:",channel)

Image loaded with the following 44 channels: ['Au', 'Background', 'Beta catenin', 'Ca', 'CD11b', 'CD11c', 'CD138', 'CD16', 'CD20', 'CD209', 'CD3', 'CD31', 'CD4', 'CD45', 'CD45RO', 'CD56', 'CD63', 'CD68', 'CD8', 'dsDNA', 'EGFR', 'Fe', 'FoxP3', 'H3K27me3', 'H3K9ac', 'HLA-DR', 'HLA_Class_1', 'IDO', 'Keratin17', 'Keratin6', 'Ki67', 'Lag3', 'MPO', 'Na', 'P', 'p53', 'Pan-Keratin', 'PD-L1', 'PD1', 'phospho-S6', 'Si', 'SMA', 'Ta', 'Vimentin']


### Step 2: Image preprocessing
##### These modules are optional. User will select what functions they want to chain together

##### OPTIONAL - Subtract Background noise

In [ ]:
index=channel.index('Au')
bgcorrect_Image=sp.background_subtract(Image, index, 10,2)

index=channel.index('Background')
bgcorrect_Image=sp.background_subtract(bgcorrect_Image, index, 10,2)

##### OPTIONAL - NLM Denoising

In [ ]:
nlm_Image=sp.nlm_denoise(bgcorrect_Image,5,6)

##### OPTIONAL - Median filter

In [5]:
list=['dsDNA','H3K9ac','H3K27me3']

to_denoise=[]      
for i in range(0,len(list),1):
    index=channel.index(list[i])
    to_denoise.append(index)
to_denoise.sort()

median_Image=sp.median_denoise(Image,4,to_denoise)

### Step 3: Cell Segmentation
##### User will select 1 of the 3 segmentation options. Output will be a label image where each cell is assigned an integer value.

##### StarDist deep learning segmentation

In [6]:
list=['dsDNA','H3K9ac','H3K27me3']

to_merge=[]      
for i in range(0,len(list),1):
    index=channel.index(list[i])
    to_merge.append(index)
to_merge.sort()

stardist_label=sp.stardist_cellseg(median_Image, to_merge, 1, 0.5, 1, 98.5)

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


##### OR Cellpose deep learning segmentation

In [7]:
list=['dsDNA','H3K9ac','H3K27me3']

to_merge=[]      
for i in range(0,len(list),1):
    index=channel.index(list[i])
    to_merge.append(index)
to_merge.sort()

cellpose_label=sp.cellpose_cellseg(median_Image, to_merge,12, 1)

2021-07-08 14:38:54,669 [INFO] >>>> using CPU
2021-07-08 14:38:56,165 [INFO] ~~~ FINDING MASKS ~~~
2021-07-08 14:42:06,196 [INFO] >>>> TOTAL TIME 190.03 sec


##### OR DeepCell deep learning segmentation

In [14]:
list=['dsDNA','H3K9ac','H3K27me3']

to_merge=[]      
for i in range(0,len(list),1):
    index=channel.index(list[i])
    to_merge.append(index)
to_merge.sort()

deepcell_label=sp.deepcell_segmentation(Image, to_merge, 0.39)

2021-07-08 15:00:18,118 [WARNING] SavedModel saved prior to TF 2.5 detected when loading Keras model. Please ensure that you are saving the model with model.save() or tf.keras.models.save_model(), *NOT* tf.saved_model.save(). To confirm, there should be a file named "keras_metadata.pb" in the SavedModel directory.
2021-07-08 15:00:26,186 [WARNING] No training configuration found in save file, so the model was *not* compiled. Compile it manually.


##### OR Classic watershed segmentation

In [ ]:
list=['dsDNA','H3K9ac','H3K27me3']

to_merge=[]      
for i in range(0,len(list),1):
    index=channel.index(list[i])
    to_merge.append(index)
to_merge.sort()

classic_label=sp.classicwatershed_cellseg(median_Image, to_merge)

### Step 4: Postprocessing
##### These modules are optional. User will select what functions they want to chain together

##### Rescue cells missed by DL model. (uses traditional watershed) - This is an optional step to add after segmentatio step.

In [ ]:
list=['dsDNA','H3K9ac','H3K27me3']

to_merge=[]      
for i in range(0,len(list),1):
    index=channel.index(list[i])
    to_merge.append(index)
to_merge.sort()

new_label=sp.rescue_cells(Image,to_merge, stardist_label)

##### Remove small and/or large segments

In [ ]:
newlabel=sp.remove_small_objects(new_label, 8)
newlabel=sp.remove_large_objects(new_label, 75)

##### Dilate nuclei boundaries to simulate a cell

In [15]:
expanded_label=sp.simulate_cell(deepcell_label, 10)

### Step 5: Feature Extraction

##### Extract features to be passed to the phenotyping modules

In [ ]:
df=sp.feature_extraction(Image, expanded_label,channel)
df

## Example Batch Process
##### Running a folder of images through the SPEX pipeline

In [22]:
os.chdir('/gne/data/pathology/t3imagedata/Projects/H2020-428/Images/PAC0002_KerenMIBI/Converted')
files = glob.glob('*.tiff', recursive = False)
files

['TA459_multipleCores2_Run-4_Point31.tiff',
 'TA459_multipleCores2_Run-4_Point38.tiff',
 'TA459_multipleCores2_Run-4_Point26.tiff',
 'TA459_multipleCores2_Run-4_Point32.tiff',
 'TA459_multipleCores2_Run-4_Point37.tiff',
 'TA459_multipleCores2_Run-4_Point6.tiff',
 'TA459_multipleCores2_Run-4_Point41.tiff',
 'TA459_multipleCores2_Run-4_Point24.tiff',
 'TA459_multipleCores2_Run-4_Point30.tiff',
 'TA459_multipleCores2_Run-4_Point3.tiff',
 'TA459_multipleCores2_Run-4_Point12.tiff',
 'TA459_multipleCores2_Run-4_Point28.tiff',
 'TA459_multipleCores2_Run-4_Point25.tiff',
 'TA459_multipleCores2_Run-4_Point18.tiff',
 'TA459_multipleCores2_Run-4_Point40.tiff',
 'TA459_multipleCores2_Run-4_Point17.tiff',
 'TA459_multipleCores2_Run-4_Point29.tiff',
 'TA459_multipleCores2_Run-4_Point15.tiff',
 'TA459_multipleCores2_Run-4_Point34.tiff',
 'TA459_multipleCores2_Run-4_Point22.tiff',
 'TA459_multipleCores2_Run-4_Point16.tiff',
 'TA459_multipleCores2_Run-4_Point1.tiff',
 'TA459_multipleCores2_Run-4_Point9

In [26]:
for image in files:
    #Load Image
    Image, channel=sp.load_tiff(image,is_mibi=True)

    #Denoise Image
    list=['dsDNA','H3K9ac','H3K27me3']

    to_denoise=[]      
    for i in range(0,len(list),1):
        index=channel.index(list[i])
        to_denoise.append(index)
    to_denoise.sort()

    median_Image=sp.median_denoise(Image,5,to_denoise)

    #Run Segmentation
    list=['dsDNA','H3K9ac','H3K27me3']

    to_merge=[]      
    for i in range(0,len(list),1):
        index=channel.index(list[i])
        to_merge.append(index)
    to_merge.sort()
    
    stardist_label=sp.stardist_cellseg(median_Image, to_merge, 1, 0.5, 1, 98.5)
    #cellpose_label=sp.cellpose_cellseg(median_Image, index,12, 1)
    deepcell_label=sp.deepcell_segmentation(Image, to_merge, 0.39)

    #index=channel.index('H3K27me3')
    new_label=sp.rescue_cells(Image,to_merge, stardist_label)
    #new_label2=sp.rescue_cells(Image,index, cellpose_label)
    #new_label3=sp.rescue_cells(Image,index, deepcell_label)

    #Dilate Cells
    expanded_label=sp.simulate_cell(new_label, 10)
    #expanded_label2=sp.simulate_cell(new_label2, 10)
    expanded_label3=sp.simulate_cell(deepcell_label, 8)

    #Extract Features
    df=sp.feature_extraction(Image, expanded_label,channel)
    df2=sp.feature_extraction(Image, expanded_label3,channel)

    #Save Feature Data
    csvname = image.split(".tiff")[0]+'_stardist.csv'
    df.to_csv(csvname, index = False)
    csvname = image.split(".tiff")[0]+'_deepcell.csv'
    df2.to_csv(csvname, index = False)

    #Save Image of segmentation
    imagename = image.split(".tiff")[0]+'_label.ome.tiff'

    contour=segmentation.find_boundaries(expanded_label, connectivity=1, mode='thick', background=0)
    #contour2=segmentation.find_boundaries(expanded_label2, connectivity=1, mode='thick', background=0)
    contour3=segmentation.find_boundaries(expanded_label3, connectivity=1, mode='thick', background=0)
    pseudoIF=np.stack((Image[channel.index('dsDNA')],Image[channel.index('H3K9ac')],Image[channel.index('H3K27me3')],contour,contour3), axis=0)

    with TiffWriter(imagename, bigtiff=True) as tif:
        options = dict(tile=(512, 512),photometric='minisblack')
        tif.write(pseudoIF, **options, metadata={'PhysicalSizeX':0.39,'PhysicalSizeY':0.39,'Channel': {'Name': ["dsDNA","H3K9ac","H3K27me3", "Stardist","DeepCell"]}})

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.
2021-07-08 15:24:28,077 [WARNING] SavedModel saved prior to TF 2.5 detected when loading Keras model. Please ensure that you are saving the model with model.save() or tf.keras.models.save_model(), *NOT* tf.saved_model.save(). To confirm, there should be a file named "keras_metadata.pb" in the SavedModel directory.
2021-07-08 15:24:37,067 [WARNING] No training configuration found in save file, so the model was *not* compiled. Compile it manually.


spex_segment.py (413): indices argument is deprecated and will be removed in version 0.20. To avoid this warning, please do not use the indices argument. Please see peak_local_max documentation for more details.


Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.
2021-07-08 15:25:57,597 [WARNING] SavedModel saved prior to TF 2.5 detected when loading Keras model. Please ensure that you are saving the model with model.save() or tf.keras.models.save_model(), *NOT* tf.saved_model.save(). To confirm, there should be a file named "keras_metadata.pb" in the SavedModel directory.
2021-07-08 15:26:05,914 [WARNING] No training configuration found in save file, so the model was *not* compiled. Compile it manually.


spex_segment.py (413): indices argument is deprecated and will be removed in version 0.20. To avoid this warning, please do not use the indices argument. Please see peak_local_max documentation for more details.


Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.
2021-07-08 15:27:24,279 [WARNING] SavedModel saved prior to TF 2.5 detected when loading Keras model. Please ensure that you are saving the model with model.save() or tf.keras.models.save_model(), *NOT* tf.saved_model.save(). To confirm, there should be a file named "keras_metadata.pb" in the SavedModel directory.
2021-07-08 15:27:32,900 [WARNING] No training configuration found in save file, so the model was *not* compiled. Compile it manually.


...

spex_segment.py (413): indices argument is deprecated and will be removed in version 0.20. To avoid this warning, please do not use the indices argument. Please see peak_local_max documentation for more details.


Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.
2021-07-08 16:29:35,980 [WARNING] SavedModel saved prior to TF 2.5 detected when loading Keras model. Please ensure that you are saving the model with model.save() or tf.keras.models.save_model(), *NOT* tf.saved_model.save(). To confirm, there should be a file named "keras_metadata.pb" in the SavedModel directory.
2021-07-08 16:29:44,174 [WARNING] No training configuration found in save file, so the model was *not* compiled. Compile it manually.


spex_segment.py (413): indices argument is deprecated and will be removed in version 0.20. To avoid this warning, please do not use the indices argument. Please see peak_local_max documentation for more details.


Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.
2021-07-08 16:31:28,152 [WARNING] SavedModel saved prior to TF 2.5 detected when loading Keras model. Please ensure that you are saving the model with model.save() or tf.keras.models.save_model(), *NOT* tf.saved_model.save(). To confirm, there should be a file named "keras_metadata.pb" in the SavedModel directory.
2021-07-08 16:31:36,740 [WARNING] No training configuration found in save file, so the model was *not* compiled. Compile it manually.


spex_segment.py (413): indices argument is deprecated and will be removed in version 0.20. To avoid this warning, please do not use the indices argument. Please see peak_local_max documentation for more details.
